# Linear Programming with the `ortools` Library



1. Import linear solver wrapper
2. Declare the LP solver (i.e. `MPsolver`, `GLOP`)
3. Create Variables (i.e. `solver.NumVar(lb, ub, name)`)
4. Add Constraints `solver.Add`
5. Define Objective `solver.Maximize()`
6. Invoke the solver `solver.Solve()`
7. Display the solution

### Example Problem

Maximize `3x + 4y` subject to the following constraints:

1. `x + 2y <= 14`
2. `3x -y >= 0`
3. `x - y <= 2`
4. `x,y >= 0`

In [1]:
# 1. Import solver wrapper
from ortools.linear_solver import pywraplp

# `MPSolver` is a wrapper for several different solvers, including Glop. 
# 2. Declare the GLOP solver
solver = pywraplp.Solver.CreateSolver("GLOP")

# 3. Create variables
x = solver.NumVar(0, solver.infinity(), "x")
y = solver.NumVar(0, solver.infinity(), "y")

print("Number of variables: ", solver.NumVariables())

Number of variables:  2


In [11]:
# 4. Constraints

# Constraint 0: x + 2y <= 14.
solver.Add(x + 2 * y <= 14.0)

# Constraint 1: 3x - y >= 0.
solver.Add(3 * x - y >= 0.0)

# Constraint 2: x - y <= 2.
solver.Add(x - y <= 2.0)

print("Number of constraints =", solver.NumConstraints())

Number of constraints = 3


In [2]:
# 5. Define objective

# Objective function: 3x + 4y.
# This also can accept some arbritrary function
solver.Maximize(3 * x + 4 * y)

# Alternative:
# objective = solver.Objective()
# objective.SetCoefficient(x, 3)
# objective.SetCoefficient(y, 1)
# objective.SetMaximization()

In [14]:
# 6. Invoke the solver

print(f"Solving with {solver.SolverVersion()}")
status = solver.Solve()

# 7. Display Solution

if status == pywraplp.Solver.OPTIMAL:
    print("Solution:")
    print(f"Objective value = {solver.Objective().Value():0.1f}")
    print(f"x = {x.solution_value():0.1f}")
    print(f"y = {y.solution_value():0.1f}")
else:
    print("The problem does not have an optimal solution.")

Solving with Glop solver v9.8.3296
Solution:
Objective value = 34.0
x = 6.0
y = 4.0


## Example

Here's is the example in one cell:

In [15]:
from ortools.linear_solver import pywraplp


def LinearProgrammingExample():
    """Linear programming sample."""
    # Instantiate a Glop solver, naming it LinearExample.
    solver = pywraplp.Solver.CreateSolver("GLOP")
    if not solver:
        return

    # Create the two variables and let them take on any non-negative value.
    x = solver.NumVar(0, solver.infinity(), "x")
    y = solver.NumVar(0, solver.infinity(), "y")

    print("Number of variables =", solver.NumVariables())

    # Constraint 0: x + 2y <= 14.
    solver.Add(x + 2 * y <= 14.0)

    # Constraint 1: 3x - y >= 0.
    solver.Add(3 * x - y >= 0.0)

    # Constraint 2: x - y <= 2.
    solver.Add(x - y <= 2.0)

    print("Number of constraints =", solver.NumConstraints())

    # Objective function: 3x + 4y.
    solver.Maximize(3 * x + 4 * y)

    # Solve the system.
    print(f"Solving with {solver.SolverVersion()}")
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print("Solution:")
        print(f"Objective value = {solver.Objective().Value():0.1f}")
        print(f"x = {x.solution_value():0.1f}")
        print(f"y = {y.solution_value():0.1f}")
    else:
        print("The problem does not have an optimal solution.")

    print("\nAdvanced usage:")
    print(f"Problem solved in {solver.wall_time():d} milliseconds")
    print(f"Problem solved in {solver.iterations():d} iterations")


LinearProgrammingExample()

Number of variables = 2
Number of constraints = 3
Solving with Glop solver v9.8.3296
Solution:
Objective value = 34.0
x = 6.0
y = 4.0

Advanced usage:
Problem solved in 0 milliseconds
Problem solved in 2 iterations


## Rosenbrock function example with `scipy.optimize`

Compare and contrast the syntax for `ortools` with `scipy.optimize`:

In [16]:
import numpy as np
from scipy.optimize import minimize

def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
res = minimize(rosen, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 339
         Function evaluations: 571


In [17]:
print(res.x)

[1. 1. 1. 1. 1.]


In [19]:
from scipy.optimize import linprog
c = np.array([-29.0, -45.0, 0.0, 0.0])
A_ub = np.array([[1.0, -1.0, -3.0, 0.0],
                [-2.0, 3.0, 7.0, -3.0]])
b_ub = np.array([5.0, -10.0])
A_eq = np.array([[2.0, 8.0, 1.0, 0.0],
                [4.0, 4.0, 0.0, 1.0]])
b_eq = np.array([60.0, 60.0])
x0_bounds = (0, None)
x1_bounds = (0, 5.0)
x2_bounds = (-np.inf, 0.5)  # +/- np.inf can be used instead of None
x3_bounds = (-3.0, None)
bounds = [x0_bounds, x1_bounds, x2_bounds, x3_bounds]
result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds)
print(result.message)

The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is At lower/fixed bound)


In [20]:
# Adjust x1_bounds
x1_bounds = (0, 6)
bounds = [x0_bounds, x1_bounds, x2_bounds, x3_bounds]
result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds)
print(result.message)

Optimization terminated successfully. (HiGHS Status 7: Optimal)


In [24]:
# Optimal Value
result.fun

-505.974358974359

In [25]:
# Optimal Solution
result.x

array([ 9.41025641,  5.17948718, -0.25641026,  1.64102564])